In [10]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    for i in range(1, size):
        message = comm.recv(source=i)
        print(f"Received from process {i}: {message}")
else:
    comm.send(f"Hello from process {rank}", dest=0)

In [2]:
!apt-get update
!apt-get install -y mpich
!pip install mpi4py

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,902 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,688 kB]
Ge

In [3]:
%%writefile mpi_master_worker.py
from mpi4py import MPI

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    # Master process
    print(f"Master process started. Expecting {size-1} worker messages...\n")

    for i in range(1, size):
        message = comm.recv(source=i)
        print(
            f"Received from process {message['rank']} | "
            f"Task: {message['task']} | "
            f"Result: {message['result']}"
        )

    print("\nAll worker results received.")

else:
    # Worker processes
    data_chunk = list(range(rank * 10, rank * 10 + 10))
    computed_sum = sum(data_chunk)

    message = {
        "rank": rank,
        "task": f"Data chunk {rank}",
        "result": computed_sum
    }

    comm.send(message, dest=0)

Writing mpi_master_worker.py


In [8]:
!mpiexec --allow-run-as-root --oversubscribe -n 4 python mpi_master_worker.py

Master process started. Expecting 3 worker messages...

Received from process 1 | Task: Data chunk 1 | Result: 145
Received from process 2 | Task: Data chunk 2 | Result: 245
Received from process 3 | Task: Data chunk 3 | Result: 345

All worker results received.
